# Nested 5-Fold Cross Validation For Logistic Regression On Textual Features

In [25]:
import numpy as np
import pandas as pd
import xlrd as xl
from pandas import ExcelWriter
from pandas import ExcelFile
import pprint
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from nltk.stem import WordNetLemmatizer
import re
import pickle
from operator import itemgetter
import time, datetime
from functools import partial, update_wrapper
from openpyxl import load_workbook

from sklearn.feature_extraction.text import TfidfVectorizer

from imblearn.over_sampling import SMOTE

from sklearn.pipeline import Pipeline
from imblearn.pipeline import Pipeline as Imb_Pipeline

from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import StratifiedKFold, GridSearchCV, cross_validate
from sklearn.metrics import precision_recall_fscore_support, classification_report, accuracy_score, make_scorer, confusion_matrix

pp = pprint.PrettyPrinter(indent=4)

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

#### Use spaCy parser for word tokenization of a sentence:

In [26]:
import spacy
from spacy.lang.en import English

# Load the English language model
nlp = spacy.load('en_core_web_sm')

# Create an instance of the English parser
parser = English()


#### Define stopwords as punctuation + common contractions:

In [27]:
from string import punctuation
from nltk.corpus import stopwords

stop_words = list(punctuation) + ["'s","'m","n't","'re","-","'ll",'...'] #+ stopwords.words('english')

#### Code to lemmatize and tokenize:

In [28]:
def get_lemma(item):
    return WordNetLemmatizer().lemmatize(item)

def tokenize(line):
    line_tokens = []
    tokens = parser(line)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            line_tokens.append('URL')
        elif token.orth_.startswith('@'):
            line_tokens.append('SCREEN_NAME')
        elif str(token) not in stop_words:
            line_tokens.append(get_lemma(token.lower_))
    return line_tokens

In [29]:
### Read from the pickled file
all_data = pd.read_excel('../data/combined_dataset.xlsx')

print("Size of corpus: "+str(len(all_data)))

Size of corpus: 8428


In [30]:
all_data = all_data.dropna(subset=['Text Content', 'Code'])

In [31]:
labels_to_remove = [ "Testing",'Future Plan','Issue Content Management']
all_data = all_data[~all_data['Code'].isin(labels_to_remove)]

In [32]:
X = all_data['Text Content'].values
y = all_data['Code'].values

print("Number of unique labels: "+str(len(set(y))))

labels = list(set(y))
labels.sort()

pp.pprint(labels)



Number of unique labels: 13
[   'Action on Issue',
    'Bug Reproduction',
    'Contribution and Commitment',
    'Expected Behaviour',
    'Investigation and Exploration',
    'Motivation',
    'Observed Bug Behaviour',
    'Potential New Issues and Requests',
    'Social Conversation',
    'Solution Discussion',
    'Solution Usage',
    'Task Progress',
    'Workarounds']


# Nested Cross-Validation on Logistic Regression:

In [33]:
# To be used within GridSearch
inner_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

# To be used in outer CV
outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)


## Pipeline: tfidf + SMOTE

In [34]:

pipeline = Imb_Pipeline([
    ('vect', TfidfVectorizer(tokenizer=tokenize)),
    ('smote', SMOTE()),
    ('clf', LogisticRegression())
])

### Hyperparameters to search
parameters = {
    'vect__ngram_range': ((1, 1), (1, 2)),  # unigrams or bigrams
    'clf__C': (0.01, 0.1, 1, 10),
}

## Nested Cross Validation using GridSearch

In [35]:
### Define and create the scoring functions
import nltk
nltk.download('wordnet')
def score_func(y_true, y_pred, score_index, i):
    return(precision_recall_fscore_support(y_true,y_pred)[score_index][i])

def avg_score(y_true, y_pred, score_index):
    return precision_recall_fscore_support(y_true,y_pred,average='weighted')[score_index]

def sum_support(y_true, y_pred):
    return len(y_true)

### Create partials for each of the metrics returned
score_funcs = {v: partial(score_func, score_index=k) for k, v in {0:'precision',1:'recall',2:'fscore',3:'support'}.items()}
prec_score = partial(score_func, score_index=0)
update_wrapper(prec_score,score_func)
rec_score = partial(score_func, score_index=1)
update_wrapper(rec_score,score_func)
f_score = partial(score_func, score_index=2)
update_wrapper(f_score,score_func)
support_score = partial(score_func, score_index=3)
update_wrapper(support_score,score_func)

### Create a callable scoring function for each of the metrics for each classification label
scorer = {}
for label_id in range(0,13):
    scorer['label'+str(label_id)+'_precision'] = make_scorer(prec_score, i=label_id)
    scorer['label'+str(label_id)+'_recall'] = make_scorer(rec_score, i=label_id)
    scorer['label'+str(label_id)+'_fscore'] = make_scorer(f_score, i=label_id)
    scorer['label'+str(label_id)+'_support'] = make_scorer(support_score, i=label_id)

### Create a callable scoring function for avg/total of the metrics across classification labels
scorer['avg_precision'] = make_scorer(avg_score,score_index=0)
scorer['avg_recall'] = make_scorer(avg_score,score_index=1)
scorer['avg_fscore'] = make_scorer(avg_score,score_index=2)
scorer['total_support'] = make_scorer(sum_support)


### Perform Nested cross-validation on Pipeline
start = time.time()
clf = GridSearchCV(pipeline, parameters, cv=inner_cv, scoring='f1_weighted')
clf_results = cross_validate(clf, X=X, y=y, cv=outer_cv, scoring=scorer)
print("Completed Pipeline scenario in "+ str(datetime.timedelta(seconds=(time.time()-start))))

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Completed Pipeline scenario in 0:27:26.571548


## Pipeline Results:

In [36]:

train_report = pd.DataFrame(columns=['Precision', 'Recall', 'F1-score', 'Support'])
test_report = pd.DataFrame(columns=['Precision', 'Recall', 'F1-score', 'Support'])

result_dict = {}

writer = pd.ExcelWriter('../results/combined_dataset_smote_result.xlsx')


datalength = 0

for i in range(0, 5):
    for label_id in range(0, 13):
        train_label_precision_key = 'train_label' + str(label_id) + '_precision'
        train_label_recall_key = 'train_label' + str(label_id) + '_recall'
        train_label_fscore_key = 'train_label' + str(label_id) + '_fscore'
        train_label_support_key = 'train_label' + str(label_id) + '_support'

        if train_label_precision_key in clf_results and train_label_recall_key in clf_results and \
            train_label_fscore_key in clf_results and train_label_support_key in clf_results:

            train_report.loc[labels[label_id], :] = [clf_results[train_label_precision_key][i],
                                                      clf_results[train_label_recall_key][i],
                                                      clf_results[train_label_fscore_key][i],
                                                      clf_results[train_label_support_key][i]]

        test_label_precision_key = 'test_label' + str(label_id) + '_precision'
        test_label_recall_key = 'test_label' + str(label_id) + '_recall'
        test_label_fscore_key = 'test_label' + str(label_id) + '_fscore'
        test_label_support_key = 'test_label' + str(label_id) + '_support'

        if test_label_precision_key in clf_results and test_label_recall_key in clf_results and \
            test_label_fscore_key in clf_results and test_label_support_key in clf_results:

            test_report.loc[labels[label_id], :] = [clf_results[test_label_precision_key][i],
                                                     clf_results[test_label_recall_key][i],
                                                     clf_results[test_label_fscore_key][i],
                                                     clf_results[test_label_support_key][i]]

    train_avg_precision_key = 'train_avg_precision'
    train_avg_recall_key = 'train_avg_recall'
    train_avg_fscore_key = 'train_avg_fscore'
    train_total_support_key = 'train_total_support'

    if train_avg_precision_key in clf_results and train_avg_recall_key in clf_results and \
        train_avg_fscore_key in clf_results and train_total_support_key in clf_results:

        train_report.loc['Avg/Total', :] = [clf_results[train_avg_precision_key][i],
                                             clf_results[train_avg_recall_key][i],
                                             clf_results[train_avg_fscore_key][i],
                                             clf_results[train_total_support_key][i]]

    test_avg_precision_key = 'test_avg_precision'
    test_avg_recall_key = 'test_avg_recall'
    test_avg_fscore_key = 'test_avg_fscore'
    test_total_support_key = 'test_total_support'

    if test_avg_precision_key in clf_results and test_avg_recall_key in clf_results and \
        test_avg_fscore_key in clf_results and test_total_support_key in clf_results:

        test_report.loc['Avg/Total', :] = [clf_results[test_avg_precision_key][i],
                                            clf_results[test_avg_recall_key][i],
                                            clf_results[test_avg_fscore_key][i],
                                            clf_results[test_total_support_key][i]]

    fold_index = pd.DataFrame(data=[{'Fold': 'Fold ' + str(i)}])
    fold_index.to_excel(writer, 'LTC', startrow=datalength, index=False)
    datalength += (len(fold_index) + 2)
    train_report.to_excel(writer, 'LTC', startrow=datalength)
    datalength += (len(train_report) + 2)
    test_report.to_excel(writer, 'LTC', startrow=datalength)
    datalength += (len(test_report) + 2)

    result_dict['LTC_train_' + str(i)] = train_report
    result_dict['LTC_test_' + str(i)] = test_report

    train_report = train_report.astype(float).round(2)
    test_report = test_report.astype(float).round(2)

    print("\n------------------------- FOLD " + str(i) + ": -------------------------")
    print("\nTraining Results:")
    print(train_report)
    print("\nTest Results:")
    print(test_report)

writer.close()



------------------------- FOLD 0: -------------------------

Training Results:
Empty DataFrame
Columns: [Precision, Recall, F1-score, Support]
Index: []

Test Results:
                                   Precision  Recall  F1-score  Support
Action on Issue                         0.68    0.62      0.65     21.0
Bug Reproduction                        0.58    0.53      0.55    119.0
Contribution and Commitment             0.78    0.74      0.76    131.0
Expected Behaviour                      0.19    0.16      0.18     31.0
Investigation and Exploration           0.56    0.39      0.46    152.0
Motivation                              0.33    0.28      0.30     71.0
Observed Bug Behaviour                  0.27    0.28      0.27     61.0
Potential New Issues and Requests       0.28    0.22      0.25     50.0
Social Conversation                     0.69    0.70      0.69    282.0
Solution Discussion                     0.60    0.67      0.64    590.0
Solution Usage                         

Average Test Report Across 5 Folds

In [37]:
# Initialize DataFrames to hold the averages
avg_test_report = pd.DataFrame(columns=['Avg Precision', 'Avg Recall', 'Avg F1-score', 'Total Support'], index=labels)

# Variables to calculate weighted averages
total_test_support = 0
weighted_test_precision = 0
weighted_test_recall = 0
weighted_test_f1 = 0

# Calculate averages across 5 folds for each label
for label_id in range(13):

    test_precisions = [result_dict['LTC_test_' + str(i)].loc[labels[label_id], 'Precision'] for i in range(5)]
    test_recalls = [result_dict['LTC_test_' + str(i)].loc[labels[label_id], 'Recall'] for i in range(5)]
    test_f1_scores = [result_dict['LTC_test_' + str(i)].loc[labels[label_id], 'F1-score'] for i in range(5)]
    test_supports = [result_dict['LTC_test_' + str(i)].loc[labels[label_id], 'Support'] for i in range(5)]

    # Calculate averages for each label
    avg_test_precision = np.mean(test_precisions)
    avg_test_recall = np.mean(test_recalls)
    avg_test_f1 = np.mean(test_f1_scores)
    total_test_label_support = np.sum(test_supports)

    avg_test_report.loc[labels[label_id]] = [avg_test_precision, avg_test_recall, avg_test_f1, total_test_label_support]

    # Accumulate for weighted averages

    weighted_test_precision += avg_test_precision * total_test_label_support
    weighted_test_recall += avg_test_recall * total_test_label_support
    weighted_test_f1 += avg_test_f1 * total_test_label_support
    total_test_support += total_test_label_support

# Calculate and add total weighted averages
if total_test_support > 0:
    avg_test_report.loc['Total/Avg', :] = [weighted_test_precision / total_test_support,
                                           weighted_test_recall / total_test_support,
                                           weighted_test_f1 / total_test_support,
                                           total_test_support]

# Convert to float and round to 2 decimal places
avg_test_report = avg_test_report.astype(float).round(2)

print("\nAverage Test Report Across 5 Folds:")
print(avg_test_report)



Average Test Report Across 5 Folds:
                                   Avg Precision  Avg Recall  Avg F1-score  \
Action on Issue                             0.68        0.63          0.65   
Bug Reproduction                            0.52        0.53          0.53   
Contribution and Commitment                 0.77        0.71          0.74   
Expected Behaviour                          0.29        0.19          0.23   
Investigation and Exploration               0.46        0.37          0.41   
Motivation                                  0.34        0.30          0.32   
Observed Bug Behaviour                      0.34        0.24          0.27   
Potential New Issues and Requests           0.25        0.20          0.22   
Social Conversation                         0.70        0.72          0.71   
Solution Discussion                         0.61        0.67          0.64   
Solution Usage                              0.41        0.59          0.48   
Task Progress              